In [3]:
import pandas as pd


fp = "data/2024-tornado-MergedReflectivityComposite-0.5km/metadata/tornados.csv"
df = pd.read_csv(fp)
df.head()

,Unnamed: 0,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,18,202411,5,1101,202411,5,1102,196653,1216653,IOWA,...,4.0,ESE,BELKNAP,40.8030,-92.3482,40.8062,-92.3437,A brief tornado occurred with a shower in Davi...,A brief tornado occurred south of Floris from ...,CSV
1,127,202402,27,1932,202402,27,1935,188190,1155977,ILLINOIS,...,2.0,S,INVERNESS,42.0572,-88.1184,42.0898,-88.0937,"Following summer-like, record warmth, severe t...",This tornado touched down in the area near Del...,CSV
2,128,202402,27,1934,202402,27,1937,188190,1155978,ILLINOIS,...,2.0,SSW,INVERNESS,42.0722,-88.1485,42.0994,-88.1177,"Following summer-like, record warmth, severe t...",This tornado formed near Studio Drive in South...,CSV
3,133,202402,27,1815,202402,27,1816,188190,1155954,ILLINOIS,...,1.0,W,PAW PAW AIRPORT,41.7168,-89.0537,41.7173,-89.0529,"Following summer-like, record warmth, severe t...","A brief tornado, which was sighted by storm ch...",CSV
4,140,202402,27,1917,202402,27,1919,188190,1155975,ILLINOIS,...,0.0,SE,GENEVA,41.8827,-88.3482,41.8866,-88.3262,"Following summer-like, record warmth, severe t...",This tornado touched down along Keslinger Road...,CSV


In [ ]:
# want 2 images per tornado
    # 1 prior to tornadogensis
    # 1 after tornadogensis

import os
import gzip
import shutil
from tqdm import tqdm
from pathlib import Path

BASEDIR    = "data/MergedReflectivityComposite"
EVENTS_DIR = "data/2024-tornado-MergedReflectivityComposite-0.5km/events"


for row in tqdm(df.iterrows()):

    yearmonth = str(row[1].BEGIN_YEARMONTH)
    day       = int(row[1].BEGIN_DAY)

    # prepend '0' if needed
    day = f"{day:02d}"

    subdir_name = yearmonth + day
    reflectivity_dir = Path(BASEDIR) / subdir_name

    start_time = row[1].BEGIN_TIME
    end_time   = row[1].END_TIME
    scale      = row[1].TOR_F_SCALE

    event_name = '_'.join([subdir_name, str(start_time), str(end_time), scale])

    gz_path_unzipped_target_dir = Path(EVENTS_DIR) / event_name
    gz_path_unzipped_target_dir.mkdir(parents=True, exist_ok=True)

    gz_paths = list(reflectivity_dir.glob("*.gz"))
    times_fp_dict = {}

    for p in gz_paths:
        sp   = str(p)
        time = sp.split('-')[-1].split(".")[0]
        times_fp_dict[int(time)] = sp

    times_sorted    = sorted(list(times_fp_dict.keys()))
    
    event_start_int = int(start_time)
    t_0             = None
    t_1             = None
    index           = 0

    while times_sorted[index] < event_start_int:
        index += 1

    t_0 = times_sorted[index - 1]
    t_1 = times_sorted[index + 0]

    # Unzip each file
    for gz_path in [Path(times_fp_dict[t_0]), Path(times_fp_dict[t_1])]:

        output_filename = gz_path_unzipped_target_dir / gz_path.stem  # stem removes '.gz'
        with gzip.open(gz_path, 'rb') as f_in:
            with open(output_filename, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

    print(f"Unzipped {gz_path} to {output_filename}")

    # Optional break for debugging (remove when running full script)